In [33]:
from src.config import spark, read_table

df = read_table("events")

In [47]:
df.head(n=5)

[Row(timestamp=1433221332117, visitorid=257597, event='view', itemid=355908, transactionid=None, timestamp_seconds=1433221332.117, date=datetime.date(2015, 6, 2), week='23'),
 Row(timestamp=1433224214164, visitorid=992329, event='view', itemid=248676, transactionid=None, timestamp_seconds=1433224214.164, date=datetime.date(2015, 6, 2), week='23'),
 Row(timestamp=1433221999827, visitorid=111016, event='view', itemid=318965, transactionid=None, timestamp_seconds=1433221999.827, date=datetime.date(2015, 6, 2), week='23'),
 Row(timestamp=1433221955914, visitorid=483717, event='view', itemid=253185, transactionid=None, timestamp_seconds=1433221955.914, date=datetime.date(2015, 6, 2), week='23'),
 Row(timestamp=1433221337106, visitorid=951259, event='view', itemid=367447, transactionid=None, timestamp_seconds=1433221337.106, date=datetime.date(2015, 6, 2), week='23')]

In [34]:
from pyspark.sql.functions import to_date, from_unixtime, col, date_format


df = df.withColumn("timestamp_seconds", col("timestamp")/1000)

#df.select(from_unixtime("timestamp_seconds")).show()

df = df.withColumn("date", to_date(from_unixtime("timestamp_seconds")))
df = df.withColumn("week", date_format(to_date(from_unixtime("timestamp_seconds")),"w"))

#date_format(to_date("date", "dd/MMM/yyyy"), "w")

In [57]:
grouped = df.where(df.event == "view").groupby("week", "visitorid").count().select("week", "visitorid", col('count').alias('n'))

In [63]:
"""weekly top five visitors. Look at windowing functions (iterate over groups, eg top users)"""
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, desc
grouped_ranked = grouped.withColumn("rank", dense_rank().over(Window.partitionBy("week").orderBy(desc("n"))))
top_five_customers = grouped_ranked.where(col("rank")<6).orderBy(asc("week"), asc("rank"))

In [68]:
grouped_ranked.where(col("rank")<6).orderBy(asc("week"), asc("rank")).show()

+----+---------+---+----+
|week|visitorid|  n|rank|
+----+---------+---+----+
|  38|  1150086|196|   5|
|  38|   198270|219|   4|
|  38|   994820|265|   3|
|  38|   152963|280|   2|
|  38|   565978|440|   1|
|  37|  1150086|389|   5|
|  37|   971454|426|   4|
|  37|   152963|436|   3|
|  37|   994820|453|   2|
|  37|   198270|522|   1|
|  36|   530559|316|   5|
|  36|  1150086|340|   4|
|  36|   737053|378|   3|
|  36|   994820|460|   2|
|  36|    79627|483|   1|
|  35|   247235|290|   5|
|  35|   994820|293|   4|
|  35|   152963|313|   3|
|  35|    79627|387|   2|
|  35|   645525|570|   1|
+----+---------+---+----+
only showing top 20 rows

